In [2]:
import pandas as pd
import numpy as np

def read_files():
    files = ['dataset/file_3_Mar18_Output_1_1.csv', 
             'dataset/file_3_Mar18_Output_1_2.csv', 
             'dataset/file_3_Mar18_Output_1_3.csv', 
             'dataset/file_3_Mar18_Output_1_4.csv', 
             'dataset/file_3_Mar18_Output_1_5.csv', 
             'dataset/file_3_Mar18_Output_1_6.csv', 
             'dataset/file_4_Mar18_Output_1_1.csv', 
             'dataset/file_4_Mar18_Output_1_2.csv', 
             'dataset/file_4_Mar18_Output_1_3.csv', 
             'dataset/file_4_Mar18_Output_1_4.csv', 
             'dataset/file_4_Mar18_Output_1_5.csv', 
             'dataset/file_4_Mar18_Output_1_6.csv']
    data = pd.DataFrame()
    for file in files:
        df = pd.read_csv(file)
        data = pd.concat([data, df])
    return data

In [3]:
data = read_files()

In [5]:
print(data.head(5))

      timestamp_call_key retailer_code       serial reason  \
0  240315173250 DAL1OA3M             D  10010009499     PP   
1  240315124742 DAL15XHK             E  10006021998     BA   
2  240317114237 DAL3BLK6             J  10016007830     PT   
3  240314120849 PHX6PL4U             E  10008003199     BA   
4  240314145902 DAL68RYS             I  10013002037     BA   

                    mos  resolved  no_of_accounts_with_syf_13_march  \
0              IA PP TR  resolved                               1.0   
1        IA BA TS nl TR     floor                               2.0   
2  IA AA BA MR nl PT TR  resolved                              12.0   
3                 IA BA  resolved                               2.0   
4     IA BA TS nl mm TR     floor                               6.0   

   account_balance_13_march delinquency_history_13_march  \
0                       NaN                         [00]   
1                    143.28                         [00]   
2                   